In [1]:
import config
from sqlalchemy import create_engine, text
import sqlalchemy as sqla
import requests
import traceback
import datetime
import datetime
import time
import json
from pprint import pprint
import glob
import os
import simplejson as json
from IPython.display import display
import threading

In [2]:
engine = create_engine("mysql://{}:{}@{}:{}/{}".format(config.USER, config.PASSWORD, config.URI, config.PORT, config.DB), echo=True)

Function to write station data to DB, take the station API response text as parameter

In [3]:
def stations_to_db(text):
    stations=json.loads(text)
    for station in stations:
        vals=(station.get('number'),station.get('address'),int(station.get('banking')),
        station.get('bike_stands'),int(station.get('bonus')),
        station.get('contract_name'),station.get('name'),
        station.get('position').get('lat'),
        station.get('position').get('lng'),station.get('status'),
        station.get('last_update')
        )
        engine.execute("insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",vals)
    return

Function to write station availability to database, take the station API response text as parameter

In [4]:
def stations_availability_to_db(text):
    stations=json.loads(text)
    for station in stations:
        print(station)
        vals=(station.get('number'),station.get('available_bikes'),station.get('available_bike_stands'),station.get('last_update'),station.get('status'))
        engine.execute("insert into availability values(%s,%s,%s,%s,%s)",vals)
    return

In [3]:
r=requests.get(config.STATIONS_URL,params={"apiKey":config.APIKEY,"contract":config.NAME})
stations = json.loads(r.text)
weather_station = {}
for station in stations:
    LATITUDE = station['position']['lat']
    LONGITUDE = station['position']['lng']
    weather_request=requests.get(config.WEATHER_URL,params={"latitude":LATITUDE,"longitude":LONGITUDE,"hourly":config.HOURLY,
    "daily":config.DAILY,"current_weather":"true","timeformat":"unixtime","timezone":config.TIMEZONE})
    weather_dict=json.loads(weather_request.text)
    weather_station[station['number']]=weather_dict
print(weather_station)
for station in weather_station:
    last_update = weather_station.get(station).get('current_weather').get('time')
    temperature = weather_station.get(station).get('current_weather').get('temperature')
    weathercode = weather_station.get(station).get('current_weather').get('weathercode')
    windspeed = weather_station.get(station).get('current_weather').get('windspeed')        
    vals_current = (station, last_update, temperature, weathercode, windspeed)
    engine.execute("insert into weather_current values(%s,%s,%s,%s,%s)",vals_current)

    # Insert weather forecast data for next 24h for each station:        
    for i in range(0,24):
        forecast_time = weather_station.get(station).get('hourly').get('time')[i]
        temperature = weather_station.get(station).get('hourly').get('temperature_2m')[i]
        precipitation = weather_station.get(station).get('hourly').get('precipitation_probability')[i]
        weathercode = weather_station.get(station).get('hourly').get('weathercode')[i]
        windspeed = weather_station.get(station).get('hourly').get('windspeed_10m')[i]
        vals_hourly = (station, last_update, forecast_time, temperature, precipitation, weathercode, windspeed)
        engine.execute("insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)",vals_hourly)

    # Insert weather forecast data for next 7 days for each station:
    for j in range(0,7):
        last_update = weather_station.get(station).get('current_weather').get('time')
        forecast_day = weather_station.get(station).get('daily').get('time')[j]
        weathercode = weather_station.get(station).get('daily').get('weathercode')[j]
        temperature_max = weather_station.get(station).get('daily').get('temperature_2m_max')[j]
        temperature_min = weather_station.get(station).get('daily').get('temperature_2m_min')[j]
        vals_daily = (station, last_update, forecast_day, weathercode, temperature_max, temperature_min)
        engine.execute("insert into weather_forecast_7d values(%s,%s,%s,%s,%s,%s)",vals_daily)    


{42: {'latitude': 53.35, 'longitude': -6.28, 'generationtime_ms': 1.1690855026245117, 'utc_offset_seconds': 0, 'timezone': 'Europe/London', 'timezone_abbreviation': 'GMT', 'elevation': 15.0, 'current_weather': {'temperature': 4.7, 'windspeed': 15.5, 'winddirection': 269.0, 'weathercode': 3, 'time': 1678060800}, 'hourly_units': {'time': 'unixtime', 'temperature_2m': '°C', 'precipitation_probability': '%', 'weathercode': 'wmo code', 'windspeed_10m': 'km/h'}, 'hourly': {'time': [1678060800, 1678064400, 1678068000, 1678071600, 1678075200, 1678078800, 1678082400, 1678086000, 1678089600, 1678093200, 1678096800, 1678100400, 1678104000, 1678107600, 1678111200, 1678114800, 1678118400, 1678122000, 1678125600, 1678129200, 1678132800, 1678136400, 1678140000, 1678143600, 1678147200, 1678150800, 1678154400, 1678158000, 1678161600, 1678165200, 1678168800, 1678172400, 1678176000, 1678179600, 1678183200, 1678186800, 1678190400, 1678194000, 1678197600, 1678201200, 1678204800, 1678208400, 1678212000, 167

2023-03-06 00:39:50,470 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-03-06 00:39:50,473 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 00:39:50,714 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-03-06 00:39:50,716 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 00:39:50,837 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-03-06 00:39:50,839 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 00:39:51,068 INFO sqlalchemy.engine.Engine insert into weather_current values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:39:51,069 INFO sqlalchemy.engine.Engine [raw sql] (42, 1678060800, 4.7, 3, 15.5)
2023-03-06 00:39:51,070 INFO sqlalchemy.engine.Engine ROLLBACK


ProgrammingError: (MySQLdb._exceptions.ProgrammingError) not enough arguments for format string
[SQL: insert into weather_current values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)]
[parameters: (42, 1678060800, 4.7, 3, 15.5)]
(Background on this error at: https://sqlalche.me/e/14/f405)

Function to write station weather to database, take the station API response text as parameter

In [ ]:
def weather_to_db(text):
    stations = json.loads(text)
    weather_station = {}
    for station in stations:
        LATITUDE = station['position']['lat']
        LONGITUDE = station['position']['lng']
        weather_request=requests.get(config.WEATHER_URL,params={"latitude":LATITUDE,"longitude":LONGITUDE,"hourly":config.HOURLY,
        "daily":config.DAILY,"current_weather":"true","timeformat":"unixtime","timezone":config.TIMEZONE})
        weather_dict=json.loads(weather_request.text)
        weather_station[station['number']]=weather_dict
    print(weather_dict)
    
    # Insert current weather data to database: 
    for station in weather_station:
        last_update = weather_station.get(station).get('current_weather').get('time')
        temperature = weather_station.get(station).get('current_weather').get('temperature')
        weathercode = weather_station.get(station).get('current_weather').get('weathercode')
        windspeed = weather_station.get(station).get('current_weather').get('windspeed')        
        vals_current = (station, last_update, temperature, weathercode, windspeed)
        engine.execute("insert into weather_current values(%s,%s,%s,%s,%s)",vals_current)
        
        # Insert weather forecast data for next 24h for each station:        
        for i in range(0,24):
            forecast_time = weather_station.get(station).get('hourly').get('time')[i]
            temperature = weather_station.get(station).get('hourly').get('temperature_2m')[i]
            precipitation = weather_station.get(station).get('hourly').get('precipitation_probability')[i]
            weathercode = weather_station.get(station).get('hourly').get('weathercode')[i]
            windspeed = weather_station.get(station).get('hourly').get('windspeed_10m')[i]
            vals_hourly = (station, last_update, forecast_time, temperature, precipitation, weathercode, windspeed)
            engine.execute("insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)",vals_hourly)
    
        # Insert weather forecast data for next 7 days for each station:
        for j in range(0,7):
            last_update = weather_station.get(station).get('current_weather').get('time')
            forecast_day = weather_station.get(station).get('daily').get('time')[j]
            weathercode = weather_station.get(station).get('daily').get('weathercode')[j]
            temperature_max = weather_station.get(station).get('daily').get('temperature_2m_max')[j]
            temperature_min = weather_station.get(station).get('daily').get('temperature_2m_min')[j]
            vals_daily = (station, last_update, forecast_day, weathercode, temperature_max, temperature_min)
            engine.execute("insert into weather_forecast_7d values(%s,%s,%s,%s,%s,%s)",vals_daily)
#         break
    return
r = requests.get(config.STATIONS_URL,params={"apiKey":config.APIKEY,"contract":config.NAME})
weather_to_db(r.text)

{'latitude': 53.35, 'longitude': -6.28, 'generationtime_ms': 0.8831024169921875, 'utc_offset_seconds': 0, 'timezone': 'Europe/London', 'timezone_abbreviation': 'GMT', 'elevation': 12.0, 'current_weather': {'temperature': 4.7, 'windspeed': 15.5, 'winddirection': 269.0, 'weathercode': 3, 'time': 1678060800}, 'hourly_units': {'time': 'unixtime', 'temperature_2m': '°C', 'precipitation_probability': '%', 'weathercode': 'wmo code', 'windspeed_10m': 'km/h'}, 'hourly': {'time': [1678060800, 1678064400, 1678068000, 1678071600, 1678075200, 1678078800, 1678082400, 1678086000, 1678089600, 1678093200, 1678096800, 1678100400, 1678104000, 1678107600, 1678111200, 1678114800, 1678118400, 1678122000, 1678125600, 1678129200, 1678132800, 1678136400, 1678140000, 1678143600, 1678147200, 1678150800, 1678154400, 1678158000, 1678161600, 1678165200, 1678168800, 1678172400, 1678176000, 1678179600, 1678183200, 1678186800, 1678190400, 1678194000, 1678197600, 1678201200, 1678204800, 1678208400, 1678212000, 16782156

2023-03-06 00:44:14,545 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:14,789 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:44:14,790 INFO sqlalchemy.engine.Engine [raw sql] (42, 1678060800, 1678086000, 4.4, 0, 3, 15.0)
2023-03-06 00:44:14,903 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:15,163 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:44:15,164 INFO sqlalchemy.engine.Engine [raw sql] (42, 1678060800, 1678089600, 4.6, 0, 3, 13.7)
2023-03-06 00:44:15,274 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:15,499 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:44:15,500 INFO sqlalchemy.engine.Engine [raw sql] (42, 1678060800, 1678093200, 5.0, 0, 3, 12.6)
2023-03-06 00:44:15,604 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:15,827 INFO sqlalchemy.engine.Engine insert into weather_fore

2023-03-06 00:44:24,748 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:24,985 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:44:24,986 INFO sqlalchemy.engine.Engine [raw sql] (30, 1678060800, 1678075200, 4.1, 0, 3, 13.0)
2023-03-06 00:44:25,097 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:25,352 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:44:25,353 INFO sqlalchemy.engine.Engine [raw sql] (30, 1678060800, 1678078800, 4.0, 0, 3, 12.6)
2023-03-06 00:44:25,466 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:25,703 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:44:25,705 INFO sqlalchemy.engine.Engine [raw sql] (30, 1678060800, 1678082400, 4.1, 0, 3, 12.6)
2023-03-06 00:44:25,824 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:26,051 INFO sqlalchemy.engine.Engine insert into weather_fore

2023-03-06 00:44:35,187 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:35,444 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:44:35,446 INFO sqlalchemy.engine.Engine [raw sql] (54, 1678060800, 1678064400, 4.8, 0, 3, 14.8)
2023-03-06 00:44:35,553 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:35,793 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:44:35,795 INFO sqlalchemy.engine.Engine [raw sql] (54, 1678060800, 1678068000, 4.8, 0, 3, 15.9)
2023-03-06 00:44:35,924 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:36,146 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:44:36,148 INFO sqlalchemy.engine.Engine [raw sql] (54, 1678060800, 1678071600, 4.5, 0, 3, 15.2)
2023-03-06 00:44:36,258 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:36,504 INFO sqlalchemy.engine.Engine insert into weather_fore

2023-03-06 00:44:45,409 INFO sqlalchemy.engine.Engine [raw sql] (54, 1678060800, 1678492800, 61, 12.6, 9.0)
2023-03-06 00:44:45,516 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:45,749 INFO sqlalchemy.engine.Engine insert into weather_forecast_7d values(%s,%s,%s,%s,%s,%s)
2023-03-06 00:44:45,750 INFO sqlalchemy.engine.Engine [raw sql] (54, 1678060800, 1678579200, 61, 12.0, 8.3)
2023-03-06 00:44:45,868 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:46,096 INFO sqlalchemy.engine.Engine insert into weather_current values(%s,%s,%s,%s,%s)
2023-03-06 00:44:46,098 INFO sqlalchemy.engine.Engine [raw sql] (108, 1678060800, 4.2, 3, 15.1)
2023-03-06 00:44:46,216 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:44:46,455 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:44:46,458 INFO sqlalchemy.engine.Engine [raw sql] (108, 1678060800, 1678060800, 4.2, 0, 3, 15.1)
2023-03-06 00:44:46,592 INFO sqlalchemy.engine.Engine COMMIT


...

2023-03-06 01:00:02,865 INFO sqlalchemy.engine.Engine [raw sql] (66, 1678060800, 1678129200, 6.1, 27, 51, 9.5)
2023-03-06 01:00:02,997 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:00:03,274 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:00:03,276 INFO sqlalchemy.engine.Engine [raw sql] (66, 1678060800, 1678132800, 5.3, 18, 2, 13.6)
2023-03-06 01:00:03,890 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:00:04,923 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:00:04,924 INFO sqlalchemy.engine.Engine [raw sql] (66, 1678060800, 1678136400, 4.5, 10, 3, 12.9)
2023-03-06 01:00:05,294 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:00:05,609 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:00:05,612 INFO sqlalchemy.engine.Engine [raw sql] (66, 1678060800, 1678140000, 4.0, 7, 3, 12.3)
2023-03-06 01:00:05,738 IN

2023-03-06 01:00:22,655 INFO sqlalchemy.engine.Engine [raw sql] (104, 1678060800, 1678118400, 6.5, 78, 53, 13.0)
2023-03-06 01:00:22,810 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:00:23,036 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:00:23,037 INFO sqlalchemy.engine.Engine [raw sql] (104, 1678060800, 1678122000, 6.2, 57, 51, 13.7)
2023-03-06 01:00:23,140 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:00:23,424 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:00:23,426 INFO sqlalchemy.engine.Engine [raw sql] (104, 1678060800, 1678125600, 6.1, 35, 3, 5.6)
2023-03-06 01:00:24,000 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:00:24,344 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:00:24,346 INFO sqlalchemy.engine.Engine [raw sql] (104, 1678060800, 1678129200, 5.5, 27, 3, 7.4)
2023-03-06 01:00:24,5

2023-03-06 01:01:25,792 INFO sqlalchemy.engine.Engine [raw sql] (101, 1678060800, 1678107600, 8.2, 44, 3, 12.5)
2023-03-06 01:01:25,899 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:01:26,114 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:01:26,115 INFO sqlalchemy.engine.Engine [raw sql] (101, 1678060800, 1678111200, 8.0, 72, 3, 11.4)
2023-03-06 01:01:26,221 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:01:26,437 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:01:26,439 INFO sqlalchemy.engine.Engine [raw sql] (101, 1678060800, 1678114800, 8.2, 100, 51, 13.5)
2023-03-06 01:01:26,542 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:01:26,756 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:01:26,757 INFO sqlalchemy.engine.Engine [raw sql] (101, 1678060800, 1678118400, 6.6, 78, 53, 12.3)
2023-03-06 01:01:2

2023-03-06 01:01:35,591 INFO sqlalchemy.engine.Engine [raw sql] (115, 1678060800, 1678096800, 5.7, 5, 3, 13.2)
2023-03-06 01:01:35,692 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:01:35,909 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:01:35,912 INFO sqlalchemy.engine.Engine [raw sql] (115, 1678060800, 1678100400, 7.3, 11, 3, 12.4)
2023-03-06 01:01:36,015 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:01:36,235 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:01:36,236 INFO sqlalchemy.engine.Engine [raw sql] (115, 1678060800, 1678104000, 8.0, 16, 3, 11.7)
2023-03-06 01:01:36,342 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:01:36,563 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:01:36,565 INFO sqlalchemy.engine.Engine [raw sql] (115, 1678060800, 1678107600, 8.2, 44, 3, 13.0)
2023-03-06 01:01:36,66

2023-03-06 01:01:45,564 INFO sqlalchemy.engine.Engine [raw sql] (47, 1678060800, 1678086000, 4.5, 0, 3, 14.9)
2023-03-06 01:01:45,672 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:01:45,892 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:01:45,893 INFO sqlalchemy.engine.Engine [raw sql] (47, 1678060800, 1678089600, 4.8, 0, 3, 14.4)
2023-03-06 01:01:45,996 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:01:46,212 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:01:46,215 INFO sqlalchemy.engine.Engine [raw sql] (47, 1678060800, 1678093200, 5.2, 0, 3, 13.4)
2023-03-06 01:01:46,325 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 01:01:46,560 INFO sqlalchemy.engine.Engine insert into weather_forecast_24h values(%s,%s,%s,%s,%s,%s,%s)
2023-03-06 01:01:46,561 INFO sqlalchemy.engine.Engine [raw sql] (47, 1678060800, 1678096800, 5.7, 5, 3, 13.2)
2023-03-06 01:01:46,671 INFO 

In [4]:
def every_5_min():
    while True:
#         try:
        now = datetime.datetime.now()
        r=requests.get(config.STATIONS_URL,params={"apiKey":config.APIKEY,"contract":config.NAME})
        stations_availability_to_db(r.text)
        time.sleep(5*60)
#         except:
            # if there is any problem, print the traceback
#             traceback.print_exception(*exc_info)
#             print(traceback.format_exc())
    return

def every_hour():
    while True:
#         try: 
        now = datetime.datetime.now()
        r=requests.get(config.STATIONS_URL,params={"apiKey":config.APIKEY,"contract":config.NAME})
        weather_to_db(r.text)
        time.sleep(60*60)
#         except:
            # if there is any problem, print the traceback
#             traceback.print_exception(*exc_info)
#             print(traceback.format_exc())
    return

def every_day():
    while True:
#         try: 
        now = datetime.datetime.now()
        r=requests.get(config.STATIONS_URL,params={"apiKey":config.APIKEY,"contract":config.NAME})
        stations_to_db(r.text)
        time.sleep(24*60*60)
#         except:
            # if there is any problem, print the traceback
#             traceback.print_exception(*exc_info)
#             print(traceback.format_exc())
    return

def main():
    thread1 = threading.Thread(target=every_5_min)
    thread1.start()

    thread2 = threading.Thread(target=every_hour)
    thread2.start()

    thread3 = threading.Thread(target=every_day)
    thread3.start()

In [7]:
if __name__ == "__main__":
    main()

{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 1, 'available_bikes': 29, 'status': 'OPEN', 'last_update': 1678060346000}
2023-03-06 00:00:46,930 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-03-06 00:00:46,933 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 00:00:47,140 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-03-06 00:00:47,143 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 00:00:47,247 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-03-06 00:00:47,250 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 00:00:47,459 INFO sqlalchemy.engine.Engine insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:00:47,461 INFO sqlalchemy.engine.Engine [raw sql] (42, 'Smithfield North', 0, 30, 0, 'dublin', 'SMITHFIELD NORTH', 53.349562, -6.278198, 

2023-03-06 00:00:50,398 INFO sqlalchemy.engine.Engine [raw sql] (32, 1, 29, 1678060283000, 'OPEN')
2023-03-06 00:00:50,501 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:00:50,619 INFO sqlalchemy.engine.Engine insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:00:50,620 INFO sqlalchemy.engine.Engine [raw sql] (52, 'York Street East', 0, 32, 0, 'dublin', 'YORK STREET EAST', 53.338755, -6.262003, 'OPEN', 1678060653000)
2023-03-06 00:00:50,729 INFO sqlalchemy.engine.Engine COMMIT
{'number': 52, 'contract_name': 'dublin', 'name': 'YORK STREET EAST', 'address': 'York Street East', 'position': {'lat': 53.338755, 'lng': -6.262003}, 'banking': False, 'bonus': False, 'bike_stands': 32, 'available_bike_stands': 9, 'available_bikes': 23, 'status': 'OPEN', 'last_update': 1678060653000}
2023-03-06 00:00:50,736 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s,%s)
2023-03-06 00:00:50,737 INFO sqlalchemy.engine.Engine [raw sql] (52, 23, 9, 167806065

2023-03-06 00:00:53,812 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:00:53,916 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:00:54,041 INFO sqlalchemy.engine.Engine insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:00:54,043 INFO sqlalchemy.engine.Engine [raw sql] (106, 'Rathdown Road', 0, 40, 0, 'dublin', 'RATHDOWN ROAD', 53.35893, -6.280337, 'OPEN', 1678060800000)
{'number': 106, 'contract_name': 'dublin', 'name': 'RATHDOWN ROAD', 'address': 'Rathdown Road', 'position': {'lat': 53.35893, 'lng': -6.280337}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 37, 'available_bikes': 3, 'status': 'OPEN', 'last_update': 1678060800000}
2023-03-06 00:00:54,144 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s,%s)
2023-03-06 00:00:54,145 INFO sqlalchemy.engine.Engine [raw sql] (106, 3, 37, 1678060800000, 'OPEN')
2023-03-06 00:00:54,173 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:00:54,263 INFO sqlalc

2023-03-06 00:00:56,921 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:00:56,991 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:00:57,139 INFO sqlalchemy.engine.Engine insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:00:57,140 INFO sqlalchemy.engine.Engine [raw sql] (45, 'Deverell Place', 0, 30, 0, 'dublin', 'DEVERELL PLACE', 53.351464, -6.255265, 'OPEN', 1678060444000)
{'number': 45, 'contract_name': 'dublin', 'name': 'DEVERELL PLACE', 'address': 'Deverell Place', 'position': {'lat': 53.351464, 'lng': -6.255265}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 14, 'available_bikes': 16, 'status': 'OPEN', 'last_update': 1678060444000}
2023-03-06 00:00:57,203 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s,%s)
2023-03-06 00:00:57,204 INFO sqlalchemy.engine.Engine [raw sql] (45, 16, 14, 1678060444000, 'OPEN')
2023-03-06 00:00:57,245 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:00:57,312 INFO s

2023-03-06 00:00:59,921 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:00:59,984 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:01:00,431 INFO sqlalchemy.engine.Engine insert into station values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
2023-03-06 00:01:00,434 INFO sqlalchemy.engine.Engine [raw sql] (116, 'Broadstone', 0, 30, 0, 'dublin', 'BROADSTONE', 53.3547, -6.272314, 'OPEN', 1678060572000)
{'number': 116, 'contract_name': 'dublin', 'name': 'BROADSTONE', 'address': 'Broadstone', 'position': {'lat': 53.3547, 'lng': -6.272314}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 26, 'available_bikes': 4, 'status': 'OPEN', 'last_update': 1678060572000}
2023-03-06 00:01:00,707 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s,%s)
2023-03-06 00:01:00,708 INFO sqlalchemy.engine.Engine [raw sql] (116, 4, 26, 1678060572000, 'OPEN')
2023-03-06 00:01:00,718 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 00:01:00,811 INFO sqlalchemy.engine.En

...

{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 0, 'available_bikes': 30, 'status': 'OPEN', 'last_update': 1678061040000}
2023-03-06 00:07:15,123 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s,%s)
2023-03-06 00:07:15,124 INFO sqlalchemy.engine.Engine [raw sql] (42, 30, 0, 1678061040000, 'OPEN')
2023-03-06 00:07:15,232 INFO sqlalchemy.engine.Engine ROLLBACK
Traceback (most recent call last):
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 1819, in _execute_context
    self.dialect.do_execute(
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 732, in do_execute
    cursor.execute(statement, parameters)
  File "/Users/minhlynguyen/opt/anaconda3/e

2023-03-06 00:07:16,179 INFO sqlalchemy.engine.Engine [raw sql] (42, 30, 0, 1678061040000, 'OPEN')
2023-03-06 00:07:16,287 INFO sqlalchemy.engine.Engine ROLLBACK
Traceback (most recent call last):
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 1819, in _execute_context
    self.dialect.do_execute(
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 732, in do_execute
    cursor.execute(statement, parameters)
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/MySQLdb/cursors.py", line 206, in execute
    res = self._query(query)
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/MySQLdb/cursors.py", line 319, in _query
    db.query(q)
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/MySQLdb/connections.py", line 259, in query
    _mysql.connection

2023-03-06 00:07:17,250 INFO sqlalchemy.engine.Engine [raw sql] (42, 30, 0, 1678061040000, 'OPEN')
2023-03-06 00:07:17,352 INFO sqlalchemy.engine.Engine ROLLBACK
Traceback (most recent call last):
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 1819, in _execute_context
    self.dialect.do_execute(
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 732, in do_execute
    cursor.execute(statement, parameters)
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/MySQLdb/cursors.py", line 206, in execute
    res = self._query(query)
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/MySQLdb/cursors.py", line 319, in _query
    db.query(q)
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/MySQLdb/connections.py", line 259, in query
    _mysql.connection

{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 0, 'available_bikes': 30, 'status': 'OPEN', 'last_update': 1678061040000}
2023-03-06 00:07:18,291 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s,%s)
2023-03-06 00:07:18,293 INFO sqlalchemy.engine.Engine [raw sql] (42, 30, 0, 1678061040000, 'OPEN')
2023-03-06 00:07:18,395 INFO sqlalchemy.engine.Engine ROLLBACK
Traceback (most recent call last):
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 1819, in _execute_context
    self.dialect.do_execute(
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 732, in do_execute
    cursor.execute(statement, parameters)
  File "/Users/minhlynguyen/opt/anaconda3/e

{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 0, 'available_bikes': 30, 'status': 'OPEN', 'last_update': 1678061040000}
2023-03-06 00:07:19,351 INFO sqlalchemy.engine.Engine insert into availability values(%s,%s,%s,%s,%s)
2023-03-06 00:07:19,352 INFO sqlalchemy.engine.Engine [raw sql] (42, 30, 0, 1678061040000, 'OPEN')
2023-03-06 00:07:19,456 INFO sqlalchemy.engine.Engine ROLLBACK
Traceback (most recent call last):
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 1819, in _execute_context
    self.dialect.do_execute(
  File "/Users/minhlynguyen/opt/anaconda3/envs/comp30830/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 732, in do_execute
    cursor.execute(statement, parameters)
  File "/Users/minhlynguyen/opt/anaconda3/e

# BELOW IS TESTING CODE

In [83]:
def main():
    # run forever
    while True:
        try:
            now = datetime.datetime.now()
            r=requests.get(config.STATIONS_URL,params={"apiKey":config.APIKEY,"contract":config.NAME})
            # store(json.loads(r.text))
            # print(r,now)
            # write_to_file(now,r.text)
            stations_to_db(r.text)
            stations_availability_to_db(r.text)
            pprint(json.loads(r.text))
            weather_to_db(r.text)
            # now sleep for 5 minutes
            time.sleep(5*60*12)
            # r.encoding='utf-8'
            # pprint(json.loads(r.text))
            

        except:
            # if there is any problem, print the traceback
            # traceback.print_exception(*exc_info)print(traceback.format_exc())

    return


if __name__ == "__main__":
    main()
    


Loop1 1
Loop2 2
Loop1 3
Loop1 4
Loop1 5
Loop1 6
Loop1 7
Loop1 8
Loop1 9
Loop1 10
Loop1 11
Loop1 12
Loop1 13
Loop1 14
Loop1 15
Loop1 16
Loop1 17
Loop1 18
Loop1 19
Loop1 20
Loop1 21
Loop1 22
Loop1 23
Loop1 24
Loop1 25
Loop1 26
Loop1 27
Loop1 28
Loop1 29
Loop1 30
Loop1 31
Loop1 32
Loop1 33
Loop1 34
Loop1 35
Loop1 36
Loop1 37
Loop1 38
Loop1 39
Loop1 40
Loop1 41
Loop1 42
Loop1 43
Loop1 44
Loop1 45
Loop1 46
Loop1 47
Loop1 48
Loop1 49
Loop2 50
Loop1 51
Loop2 52
Loop1 53
Loop1 54
Loop2 55
Loop1 56
Loop2 57
Loop1 58
Loop1 59
Loop1 60
Loop1 61
Loop1 62
Loop1 63
Loop1 64
Loop2 65
Loop1 66
Loop1 67
Loop1 68
Loop1 69
Loop1 70
Loop1 71
Loop1 72
Loop1Loop1 74
 73
Loop1 75
Loop1 76
Loop1 77
Loop1 78
Loop1 79
Loop1 80
Loop1 81
Loop1 82
Loop1 83
Loop1 84
Loop1 85
Loop1 86
Loop1 87
Loop1 88
Loop1 89
Loop1 90
Loop1 91
Loop1 92
Loop1 93
Loop1 94
Loop1 95
Loop1 96
Loop1 97
Loop1 98
Loop1 99
Loop1 100
Loop1 101
Loop1 102
Loop1 103
Loop1 104
Loop1 105
Loop1 106
Loop1 107
Loop1 108
Loop1 109
Loop1 110
Loop1 11

In [121]:
query_station = 1
res = engine.execute("select hourly_time from weather where station = %s",query_station)
res_dict = res.mappings().one()
# res_dict = res.fetchall()

2023-03-03 15:48:56,119 INFO sqlalchemy.engine.Engine select hourly_time from weather where station = %s
2023-03-03 15:48:56,121 INFO sqlalchemy.engine.Engine [raw sql] (1,)


In [122]:
print(type(res))
print(type(res_dict))

<class 'sqlalchemy.engine.cursor.LegacyCursorResult'>
<class 'sqlalchemy.engine.row.ROMappingView'>


In [123]:
print(res_dict)

ROMappingView({'hourly_time': '[1677852000, 1677855600, 1677859200, 1677862800, 1677866400, 1677870000, 1677873600, 1677877200, 1677880800, 1677884400, 1677888000, 1677891600, 1677895200, 1677898800, 1677902400, 1677906000, 1677909600, 1677913200, 1677916800, 1677920400, 1677924000, 1677927600, 1677931200, 1677934800]'})


In [87]:
print(res_dict)

[]


In [4]:
for station in stations:
    print(station['position']['lat'])

53.349562
53.3537415547453
53.336021
53.359405
53.336597
53.33796
53.343368
53.334123
53.344304
53.338755
53.347777
53.336074
53.330091
53.350929
53.341515
53.344603
53.340927
53.348279
53.35893
53.357841
53.344115
53.343893
53.347477
53.339005
53.344153
53.334295
53.340803
53.351464
53.333653
53.343105
53.341428
53.338614
53.341833
53.346637
53.343034
53.346867
53.3547
53.33403
53.346026
53.330662
53.342113
53.357043
53.348875
53.342081
53.346874
53.347932
53.334432
53.343565
53.359624
53.354929
53.350974
53.337132
53.339218
53.347122
53.355173
53.350291
53.354845
53.335742
53.343653
53.347884
53.341288
53.347508
53.341805
53.341645
53.342296
53.337757
53.358115
53.342638
53.339434
53.339334
53.337494
53.35561
53.343897
53.346985
53.355954
53.337824
53.346603
53.345922
53.35023
53.356307
53.359246
53.330362
53.359967
53.356769
53.343456
53.35463
53.356717
53.339764
53.34744
53.355473
53.347972
53.347692
53.344007
53.347106
53.352149
53.345203
53.353331
53.338776
53.359157
53.349013
53

In [10]:
print(weather_station)

{42: {'latitude': 53.35, 'longitude': -6.28, 'generationtime_ms': 8.638978004455566, 'utc_offset_seconds': 0, 'timezone': 'Europe/London', 'timezone_abbreviation': 'GMT', 'elevation': 15.0, 'current_weather': {'temperature': 7.8, 'windspeed': 17.6, 'winddirection': 47.0, 'weathercode': 2, 'time': 1677693600}, 'hourly_units': {'time': 'unixtime', 'temperature_2m': '°C', 'precipitation_probability': '%', 'weathercode': 'wmo code', 'windspeed_10m': 'km/h'}, 'hourly': {'time': [1677628800, 1677632400, 1677636000, 1677639600, 1677643200, 1677646800, 1677650400, 1677654000, 1677657600, 1677661200, 1677664800, 1677668400, 1677672000, 1677675600, 1677679200, 1677682800, 1677686400, 1677690000, 1677693600, 1677697200, 1677700800, 1677704400, 1677708000, 1677711600, 1677715200, 1677718800, 1677722400, 1677726000, 1677729600, 1677733200, 1677736800, 1677740400, 1677744000, 1677747600, 1677751200, 1677754800, 1677758400, 1677762000, 1677765600, 1677769200, 1677772800, 1677776400, 1677780000, 16777

In [28]:
print(weather_station[42])

{'latitude': 53.35, 'longitude': -6.28, 'generationtime_ms': 8.638978004455566, 'utc_offset_seconds': 0, 'timezone': 'Europe/London', 'timezone_abbreviation': 'GMT', 'elevation': 15.0, 'current_weather': {'temperature': 7.8, 'windspeed': 17.6, 'winddirection': 47.0, 'weathercode': 2, 'time': 1677693600}, 'hourly_units': {'time': 'unixtime', 'temperature_2m': '°C', 'precipitation_probability': '%', 'weathercode': 'wmo code', 'windspeed_10m': 'km/h'}, 'hourly': {'time': [1677628800, 1677632400, 1677636000, 1677639600, 1677643200, 1677646800, 1677650400, 1677654000, 1677657600, 1677661200, 1677664800, 1677668400, 1677672000, 1677675600, 1677679200, 1677682800, 1677686400, 1677690000, 1677693600, 1677697200, 1677700800, 1677704400, 1677708000, 1677711600, 1677715200, 1677718800, 1677722400, 1677726000, 1677729600, 1677733200, 1677736800, 1677740400, 1677744000, 1677747600, 1677751200, 1677754800, 1677758400, 1677762000, 1677765600, 1677769200, 1677772800, 1677776400, 1677780000, 1677783600

In [11]:
print(weather_dict['current_weather'])

{'temperature': 6.1, 'windspeed': 11.8, 'winddirection': 20.0, 'weathercode': 3, 'time': 1677621600}


In [13]:
print(weather_dict['hourly']['time'])

[1677542400, 1677546000, 1677549600, 1677553200, 1677556800, 1677560400, 1677564000, 1677567600, 1677571200, 1677574800, 1677578400, 1677582000, 1677585600, 1677589200, 1677592800, 1677596400, 1677600000, 1677603600, 1677607200, 1677610800, 1677614400, 1677618000, 1677621600, 1677625200, 1677628800, 1677632400, 1677636000, 1677639600, 1677643200, 1677646800, 1677650400, 1677654000, 1677657600, 1677661200, 1677664800, 1677668400, 1677672000, 1677675600, 1677679200, 1677682800, 1677686400, 1677690000, 1677693600, 1677697200, 1677700800, 1677704400, 1677708000, 1677711600, 1677715200, 1677718800, 1677722400, 1677726000, 1677729600, 1677733200, 1677736800, 1677740400, 1677744000, 1677747600, 1677751200, 1677754800, 1677758400, 1677762000, 1677765600, 1677769200, 1677772800, 1677776400, 1677780000, 1677783600, 1677787200, 1677790800, 1677794400, 1677798000, 1677801600, 1677805200, 1677808800, 1677812400, 1677816000, 1677819600, 1677823200, 1677826800, 1677830400, 1677834000, 1677837600, 167

In [14]:
print(weather_dict['hourly']['temperature_2m'])

[7.0, 7.4, 7.7, 7.1, 6.7, 5.2, 6.1, 7.1, 6.6, 7.0, 7.9, 8.0, 7.7, 7.9, 8.3, 8.2, 8.0, 7.5, 7.2, 6.6, 6.5, 6.2, 6.1, 6.8, 7.4, 7.2, 7.1, 7.2, 7.0, 6.9, 6.7, 7.4, 6.9, 7.0, 7.4, 7.3, 8.0, 7.8, 7.1, 7.4, 7.7, 7.5, 7.3, 6.9, 7.1, 6.9, 7.0, 7.3, 7.2, 7.3, 7.3, 7.2, 7.1, 7.2, 6.8, 4.4, 4.2, 5.3, 6.7, 6.9, 7.0, 7.1, 7.6, 7.7, 7.4, 6.9, 5.7, 4.5, 3.9, 3.6, 3.9, 3.5, 2.8, 2.7, 3.1, 2.8, 2.1, 2.5, 3.0, 3.3, 3.5, 4.4, 5.1, 6.0, 7.1, 7.8, 8.3, 8.6, 8.2, 7.5, 6.7, 5.5, 4.4, 3.4, 2.7, 2.6, 3.0, 2.8, 2.9, 3.0, 3.1, 3.6, 3.8, 3.5, 3.5, 3.9, 4.8, 6.1, 7.5, 7.9, 8.0, 7.9, 7.7, 7.5, 7.0, 6.4, 5.8, 4.8, 3.8, 2.8, 1.7, 1.3, 1.2, 1.1, 1.0, 1.0, 1.1, 1.2, 1.4, 2.2, 3.5, 5.1, 7.0, 7.7, 8.1, 8.2, 8.0, 7.6, 7.1, 5.3, 4.4, 3.5, 3.4, 3.6, 3.9, 4.0, 4.1, 4.2, 4.2, 4.2, 4.1, 3.9, 3.8, 3.8, 4.2, 4.7, 5.5, 5.9, 6.3, 6.5, 6.2, 5.7, 4.8, 4.1, 3.2, 2.2, 1.6, 1.1]


In [18]:
string= '[1677542400, 1677546000, 1677549600, 1677553200, 1677556800, 1677560400, 1677564000, 1677567600, 1677571200, 1677574800, 1677578400, 1677582000, 1677585600, 1677589200, 1677592800, 1677596400, 1677600000, 1677603600, 1677607200, 1677610800, 1677614400, 1677618000, 1677621600, 1677625200, 1677628800, 1677632400, 1677636000, 1677639600, 1677643200, 1677646800, 1677650400, 1677654000, 1677657600, 1677661200, 1677664800, 1677668400, 1677672000, 1677675600, 1677679200, 1677682800, 1677686400, 1677690000, 1677693600, 1677697200, 1677700800, 1677704400, 1677708000, 1677711600, 1677715200, 1677718800, 1677722400, 1677726000, 1677729600, 1677733200, 1677736800, 1677740400, 1677744000, 1677747600, 1677751200, 1677754800, 1677758400, 1677762000, 1677765600, 1677769200, 1677772800, 1677776400, 1677780000, 1677783600, 1677787200, 1677790800, 1677794400, 1677798000, 1677801600, 1677805200, 1677808800, 1677812400, 1677816000, 1677819600, 1677823200, 1677826800, 1677830400, 1677834000, 1677837600, 1677841200, 1677844800, 1677848400, 1677852000, 1677855600, 1677859200, 1677862800, 1677866400, 1677870000, 1677873600, 1677877200, 1677880800, 1677884400, 1677888000, 1677891600, 1677895200, 1677898800, 1677902400, 1677906000, 1677909600, 1677913200, 1677916800, 1677920400, 1677924000, 1677927600, 1677931200, 1677934800, 1677938400, 1677942000, 1677945600, 1677949200, 1677952800, 1677956400, 1677960000, 1677963600, 1677967200, 1677970800, 1677974400, 1677978000, 1677981600, 1677985200, 1677988800, 1677992400, 1677996000, 1677999600, 1678003200, 1678006800, 1678010400, 1678014000, 1678017600, 1678021200, 1678024800, 1678028400, 1678032000, 1678035600, 1678039200, 1678042800, 1678046400, 1678050000, 1678053600, 1678057200, 1678060800, 1678064400, 1678068000, 1678071600, 1678075200, 1678078800, 1678082400, 1678086000, 1678089600, 1678093200, 1678096800, 1678100400, 1678104000, 1678107600, 1678111200, 1678114800, 1678118400, 1678122000, 1678125600, 1678129200, 1678132800, 1678136400, 1678140000, 1678143600]'
time=json.loads(string)

In [19]:
print(type(string))
print(type(time))

<class 'str'>
<class 'list'>
